In [ ]:
import os
import re
import csv
import nltk
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from numpy.linalg import norm
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



from google.colab import drive
drive.mount('/content/drive')
random.seed(time.time())
nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def cosine_similarity(vector_a, vector_b):
    dot_product = np.dot(vector_a, vector_b)
    norm_a = norm(vector_a)
    norm_b = norm(vector_b)

    similarity = dot_product / (norm_a * norm_b)
    return similarity


def _2file_2data(list_root, list_rand, data):
    row = []
    for file_a in list_file_a:
      with open(file_a, "r") as f:
          normalize_content_a = f.read()

      normalize_content_a = normalize_content_a.replace("\n", " ")

      for file_b in list_file_b:
          with open(file_b, "r") as f:
              normalize_content_b = f.read()

          normalize_content_b = normalize_content_b.replace("\n", " ")
          row.append(normalize_content_a)
          row.append(normalize_content_b)

          if list_root == list_rand:
              row.append(1)
          else:
              row.append(0)
          data.append(row)
          row = []
    return data


In [ ]:
root_folder = '/content/drive/MyDrive/process_data/'

dict_rand_file = {}
data = []

for i in range(1,26):
    list_file_a = []
    list_file_b = []
    folder_path = f'/content/drive/MyDrive/process_data/data/{i}'
    for file_name in os.listdir(folder_path):
        if "NORMALIZE" in file_name:
            file_path = os.path.join(folder_path, file_name)
            list_file_a.append(file_path)

    random_value = random.choice([x for x in range(1, 26) if (x != i and str(i) not in dict_rand_file) or (x != i and str(i) in dict_rand_file and x != dict_rand_file[str(i)])])

    folder_path = f'/content/drive/MyDrive/process_data/data/{random_value}'
    for file_name in os.listdir(folder_path):
        if "NORMALIZE" in file_name:
            file_path = os.path.join(folder_path, file_name)
            list_file_b.append(file_path)

    dict_rand_file[str(random_value)] = str(i)

    data = _2file_2data(list_file_a, list_file_b, data)
    data = _2file_2data(list_file_b, list_file_a, data)
    data = _2file_2data(list_file_a, list_file_a, data)



In [ ]:
feature_names = ['funct1', 'funct2', 'label']

csv_file_path = '/content/drive/MyDrive/process_data/dataset.csv'

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(feature_names)
    writer.writerows(data)

df = pd.read_csv(csv_file_path, index_col=False)


In [ ]:
train, test = train_test_split(df, test_size=0.4, random_state=44)

In [ ]:
test = df.sample(frac=0.3, random_state=44)
train = df.sample(frac=1, random_state=44)

In [ ]:
combined_texts = [f"{text_feature1} {text_feature2}" for text_feature1, text_feature2 in zip(train.funct1, train.funct2)]


tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_texts)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
print(f"Word index length: {len(tokenizer.word_index)}")
print(f"Some words: {list(tokenizer.word_index.keys())[0:200]}")


Word index length: 70
Some words: ['typeone', 'typesix', 'mov', 'typethree', 'call', 'typefive', 'lea', 'typeseven', 'xor', 'push', 'pop', 'add', 'sub', 'ret', 'test', 'je', 'jmp', 'typefour', 'jne', 'endbr64', 'typetwo', 'movaps', 'nop', 'cmp', 'cs', 'movups', 'movsx', 'movsxd', 'shr', 'and', 'xorps', 'movzx', 'paddd', 'xchg', 'pxor', 'setne', 'cmove', 'jbe', 'movd', 'imul', 'punpcklwd', 'punpcklbw', 'inc', 'cdqe', 'psrad', 'sar', 'dec', 'ja', 'cdq', 'cmovns', 'neg', 'sete', 'cmovne', 'movdqa', 'or', 'shl', 'notrack', 'jb', 'idiv', 'pshufd', 'rep', 'cqo', 'pcmpgtw', 'punpckhwd', 'psrldq', 'jae', 'data16', 'movdqu', 'pcmpgtb', 'punpckhbw']


In [ ]:
max_length = -1

for tweet in df["funct1"]:
    length = len(tweet.split())
    if length > max_length:
        max_length = length

print(f"Maximum token length: {max_length}")

Maximum token length: 360


In [ ]:
sequences_feature1 = tokenizer.texts_to_sequences(train.funct1)
sequences_feature2 = tokenizer.texts_to_sequences(train.funct2)

padded_feature1 = pad_sequences(sequences_feature1, maxlen=max_length, padding='post')
padded_feature2 = pad_sequences(sequences_feature2, maxlen=max_length, padding='post')

input_feature1 = padded_feature1
input_feature2 = padded_feature2



In [ ]:
y = df.label.values

X_train_feature1, X_val_feature1, X_train_feature2, X_val_feature2, y_train, y_val = train_test_split(
    input_feature1, input_feature2, y, test_size=0.2, random_state=42
)

In [ ]:
list_word = list(tokenizer.word_index.keys())


In [ ]:
root_folder = '/content/drive/MyDrive/process_data/data/'

content = ""
for i in range(1,26):
  folder_path = f'/content/drive/MyDrive/process_data/data/{str(i)}'
  for file_name in os.listdir(folder_path):
    if "NORMALIZE" in file_name:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r") as f:
            content += f.read() + "\n\n"
tokenized_code = [word_tokenize(line.lower()) for line in content.split('\n') if line]
model = Word2Vec(sentences=tokenized_code, vector_size=300, window=5, min_count=1, workers=4, sg=1)

embedding_dimension = 300
embeddings_matrix = np.zeros((vocab_size, embedding_dimension))
for word, index in tokenizer.word_index.items():
    embeddings_matrix[index] = model.wv[word]


In [ ]:
input_feature1 = tf.keras.layers.Input(shape=(max_length,), name='funct1')
embedding_layer_feature1 = tf.keras.layers.Embedding(
    vocab_size,
    embedding_dimension,
    input_length=max_length,
    weights=[embeddings_matrix],
    trainable=False
)(input_feature1)

input_feature2 = tf.keras.layers.Input(shape=(max_length,), name='funct2')
embedding_layer_feature2 = tf.keras.layers.Embedding(
    vocab_size,
    embedding_dimension,
    input_length=max_length,
    weights=[embeddings_matrix],
    trainable=False
)(input_feature2)

concatenated_features = tf.keras.layers.Concatenate()([embedding_layer_feature1, embedding_layer_feature2])


In [ ]:
cnn_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=256, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, return_sequences=True)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="tanh"),
])

main_model = tf.keras.models.Model(inputs=[input_feature1, input_feature2 ], outputs=cnn_lstm_model(concatenated_features))

main_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

main_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 funct1 (InputLayer)         [(None, 360)]                0         []                            
                                                                                                  
 funct2 (InputLayer)         [(None, 360)]                0         []                            
                                                                                                  
 embedding_18 (Embedding)    (None, 360, 100)             7100      ['funct1[0][0]']              
                                                                                                  
 embedding_19 (Embedding)    (None, 360, 100)             7100      ['funct2[0][0]']              
                                                                                            

In [ ]:
history = main_model.fit(
    [X_train_feature1, X_train_feature2],
    y_train,
    epochs=10,
    batch_size=512,
    validation_data=([X_val_feature1, X_val_feature2], y_val),
)

Epoch 1/10
8/8 [==============================] - 13s 446ms/step - loss: 1.6356 - accuracy: 0.6656 - val_loss: 0.9747 - val_accuracy: 0.6708
Epoch 2/10
8/8 [==============================] - 1s 141ms/step - loss: 0.9227 - accuracy: 0.6656 - val_loss: 0.7922 - val_accuracy: 0.6708
Epoch 3/10
8/8 [==============================] - 1s 147ms/step - loss: 0.7746 - accuracy: 0.6656 - val_loss: 0.6975 - val_accuracy: 0.6708
Epoch 4/10
8/8 [==============================] - 1s 149ms/step - loss: 0.7064 - accuracy: 0.6615 - val_loss: 0.6658 - val_accuracy: 0.6625
Epoch 5/10
8/8 [==============================] - 1s 144ms/step - loss: 0.6800 - accuracy: 0.6404 - val_loss: 0.6669 - val_accuracy: 0.6302
Epoch 6/10
8/8 [==============================] - 1s 151ms/step - loss: 0.6817 - accuracy: 0.6201 - val_loss: 0.6662 - val_accuracy: 0.6240
Epoch 7/10
8/8 [==============================] - 1s 153ms/step - loss: 0.6803 - accuracy: 0.6263 - val_loss: 0.6603 - val_accuracy: 0.6531
Epoch 8/10
8/8 [===